<a href="https://colab.research.google.com/github/janastadtruckerova/Python-for-data-science/blob/main/AI_job_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projekt: Analýza Vplyvu AI a Automatizácie na Pracovné Pozície**

# Cieľ projektu

Tento projekt  umožní preskúmať vplyv AI a automatizácie na rôzne pracovné pozície, odvetvia a zručnosti. Budeme  identifikovať, ktoré pozície sú najviac ohrozené, ktoré vznikajú a aké zručnosti sú žiadané v AI ére.

Cieľ projektu: Preskúmať dataset, identifikovať a vizualizovať vzťahy medzi úrovňou prijatia AI, rizikom automatizácie, potrebnými zručnosťami a rastovými projekciami pre rôzne typy pracovných pozícií a odvetví.

# Dataset

Použitý dataset
AI-Powered Job Market Insights
a synthetic but realistic snapshot of the modern job market on the role of AI
link na dataset: https://www.kaggle.com/datasets/uom190346a/ai-powered-job-market-insights

# 1. Načítanie a úvodné preskúmanie dát (Pandas a EDA Základy)

In [ ]:
# Import knižníc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Načítanie dát z csv súboru a preskúmanie dát

In [ ]:
# Načítanie datasetu
df = pd.read_csv("ai_job_market_insights.csv")
# Základné info o datasete
df.info()

In [ ]:
# Prvých 5 riadkov datasetu
df.head()

# 2. Čistenie a príprava dát (Pandas)

Zmena dátových typov premenných

In [ ]:

# 1. AI_Adoption_Level
ai_adoption_order = ['Low', 'Medium', 'High']
df['AI_Adoption_Level'] = pd.Categorical(
    df['AI_Adoption_Level'],
    categories=ai_adoption_order,
    ordered=True
)
# 2. Automation_Risk
automation_risk_order = ['Low', 'Medium', 'High']
df['Automation_Risk'] = pd.Categorical(
    df['Automation_Risk'],
    categories=automation_risk_order,
    ordered=True
)
# 3. Job_Growth_Projection

job_growth_order = ['Decline', 'Stable', 'Growth', 'High Growth']
df['Job_Growth_Projection'] = pd.Categorical(
    df['Job_Growth_Projection'],
    categories=job_growth_order,
    ordered=True
)

# 4. Remote_Friendly
remote_mapping = {'Yes': True, 'No': False}
df['Remote_Friendly'] = df['Remote_Friendly'].map(remote_mapping).astype(bool)

# 5. Company_Size
company_size_order = ['Small', 'Medium', 'Large']
df['Company_Size'] = pd.Categorical(
    df['Company_Size'],
    categories=company_size_order,
    ordered=True
)

# 6. Industry, Location (voliteľné pre kategóriu, ak majú obmedzený počet unikátnych hodnôt)
df['Industry'] = df['Industry'].astype('category')
df['Location'] = df['Location'].astype('category')

# Salary_USD (už float64) a Job_Title, Required_Skills (object) sú v poriadku tak, ako sú.

print("\nNové Dtypes po spracovaní:")
print(df.info())

print("\nUkážka DataFrame po konverzii:")
print(df.head())

# 3. Exploračná analýza dát (EDA) a Vizualizácie (Matplotlib a Seaborn)

 Rozdelenie úrovne adaptácie AI

In [ ]:
# AI adaptation level

# Spočítanie výskytov
df_counts = df['AI_Adoption_Level'].value_counts()
print("\nPočet výskytov jednotlivých druhov (df_counts):\n", df_counts)

# Vizualizácia pomocou  knižnice Seaborn
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='AI_Adoption_Level', palette='viridis')
plt.title('Rozdelenie úrovne adaptácie AI')
plt.xlabel('Úroveň adaptácie AI')
plt.ylabel('Počet výskytov')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


Riziko automatizácie - Ktorá kategória rizika dominuje?

In [ ]:
# Celkové riziko automatizácie
# Spočítanie výskytov (dobrý krok pre kontrolu, Seaborn to urobí aj automaticky)
df_counts = df['Automation_Risk'].value_counts() # Opravená medzera v názve stĺpca
print("\nPočet výskytov jednotlivých úrovní rizika automatizácie (df_counts):\n", df_counts)
# Vizualizácia pomocou  knižnice Seaborn
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Automation_Risk', palette='viridis') # Opravená medzera v názve stĺpca
plt.title('Rozdelenie úrovne rizika automatizácie')
plt.xlabel('Úroveň rizika automatizácie') # Opravená medzera v názve stĺpca
plt.ylabel('Počet výskytov')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Riziko automatizácia podľa jednotlivých pracovných pozícií - Ktoré pozície sú najviac ohrozené automatizáciou?

In [ ]:
# Riziko automatizácie podľa pracovných pozícií


# Top 5 pozícií pre každú kategóriu rizika
top_n = 5 # top pozícií pre každé riziko

# Zoskupenie dát podľa rizika automatizácie a názvu pozície, počet pozícií
job_risk_counts = df.groupby(['Automation_Risk', 'Job_Title']).size().reset_index(name='Count')

# Pre každú kategóriu rizika vyberieme top 5 pozícií
top_jobs_per_risk = job_risk_counts.groupby('Automation_Risk').apply(
    lambda x: x.nlargest(top_n, 'Count')
).reset_index(drop=True)

print("Top pozície v každej kategórii rizika automatizácie:")
print(top_jobs_per_risk)
print("-" * 50)

# Vizualizácia pomocou FacetGrid

# FacetGrid
# Každá kategória rizika má vlastný podgraf s top pozíciami
plt.figure(figsize=(15, 6))
g = sns.catplot(
    data=top_jobs_per_risk,
    x='Job_Title',
    y='Count',
    col='Automation_Risk',
    kind='bar',
    col_wrap=3,
    height=4, aspect=1.2,
    palette='viridis',
    sharey=False,
    order=top_jobs_per_risk.sort_values(by=['Automation_Risk', 'Count'], ascending=[True, False]).Job_Title
)
g.set_axis_labels("Pracovná pozícia", "Počet výskytov")
g.set_titles("Riziko: {col_name}")
g.set_xticklabels(rotation=45, ha='right') # Natoč popisky X-ovej osi pre lepšiu čitateľnosť
plt.suptitle('Top pracovné pozície podľa rizika automatizácie', y=1.02, fontsize=16) # Celkový názov
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Uprav rozloženie, aby sa názvy neprekrývali
plt.show()




Riziko automatizácie podľa odvetví - Kotré odvetvia sú najviac ohrozené automatizáciou?

In [ ]:


# Top 5 odvetví pre každú kategóriu rizika
top_n = 5 # Koľko top odvetví chceš zobraziť pre každé riziko

# Zoskupenie dát podľa rizika automatizácie a odvetvia, počet odvetví v jednotlivých úrovniach
industry_risk_counts = df.groupby(['Automation_Risk', 'Industry']).size().reset_index(name='Count')

# Pre každú kategóriu rizika vyberieme top 5 odvetví
top_industry_per_risk = industry_risk_counts.groupby('Automation_Risk').apply(
    lambda x: x.nlargest(top_n, 'Count')
).reset_index(drop=True)

print("Top odvetvia v každej kategórii rizika automatizácie:")
print(top_industry_per_risk)
print("-" * 50)

# Vizualizácia pomocou FacetGrid

# Každá kategória rizika má vlastný podgraf s top odvetviami
plt.figure(figsize=(15, 6)) # Zväčši celkovú veľkosť pre viac grafov

g = sns.catplot(
    data=top_industry_per_risk,
    x='Industry',
    y='Count',
    col='Automation_Risk',
    kind='bar',
    col_wrap=3,
    height=4, aspect=1.2,
    palette='viridis',
    sharey=False,
    #Zoradenie odvetvia v rámci každého grafu podľa ich počtu
    order=top_industry_per_risk.sort_values(by=['Automation_Risk', 'Count'], ascending=[True, False])['Industry']
)

g.set_axis_labels("Odvetvie", "Počet výskytov")
g.set_titles("Riziko: {col_name}")
g.set_xticklabels(rotation=45, ha='right')
plt.suptitle('Top odvetvia podľa rizika automatizácie', y=1.02, fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.98])
plt.show()

Rast pozícií - Sú firmy s vššou mierou adaptácie AI - vyšší rast pozícií

In [ ]:

# Vytvorenie stĺpcového grafu

plt.figure(figsize=(12, 7))
sns.countplot(
    data=df,
    x='AI_Adoption_Level',
    hue='Job_Growth_Projection',
    palette='viridis',
    edgecolor='black'
)

plt.title('Rozdelenie projekcie rastu pozícií podľa úrovne adaptácie AI', fontsize=16)
plt.xlabel('Úroveň adaptácie AI', fontsize=12)
plt.ylabel('Počet pozícií', fontsize=12)

plt.grid(axis='y', linestyle='--', alpha=0.7) # Pridá vodorovnú mriežku
plt.legend(title='Projekcia rastu', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

Najčastejšie požadované zručnosti

In [ ]:
from collections import Counter

#1. Extrahovanie a spočítanie jednotlivých zručností (jednoduchšie)

# Prevedie stĺpec na malé písmená a rozdelí ho.
# .str.lower(): Všetko na malé písmená.
# .str.split(',\s*'): Rozdelí reťazec podľa čiarky a ľubovoľného počtu medzier za ňou.
# .explode(): Prevedie zoznamy v bunkách na samostatné riadky.
# .str.strip(): Odstráni prípadné medzery na začiatku/konci každej zručnosti.
all_skills = df['Required_Skills'].str.lower().str.split(',\s*').explode().str.strip()

# Odstránenie prázdnych reťazccou, ktoré mohli vzniknúť, ak bol niekde "  ,  "
all_skills = all_skills[all_skills != '']

# Odstránenie NaN hodnôt
all_skills = all_skills.dropna()

# Frekvencia zručností
skill_counts = Counter(all_skills)

# Konvertovanie na DataFrame pre vizualizáciu
skill_counts_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Count'])

# Zoradenie od najčastejšej po najmenej častú hodnotu
skill_counts_df = skill_counts_df.sort_values(by='Count', ascending=False)

print("Top 20 najčastejších zručností (jednoduchý prístup):")
print(skill_counts_df.head(20))

# 2. Vizualizácia Top 5 zručností

top_n_skills = 20 # Koľko najčastejších zručností chceme zobraziť
most_common_skills = skill_counts_df.head(top_n_skills)

plt.figure(figsize=(12, 8))

sns.barplot(
    data=most_common_skills,
    x='Count',
    y='Skill',
    palette='viridis',
    edgecolor='black'
)

plt.title(f'Top {top_n_skills} najčastejšie požadovaných zručností', fontsize=16)
plt.xlabel('Počet výskytov', fontsize=12)
plt.ylabel('Zručnosť', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


Korelácie

Vzťah medzi AI_Adoption_Level a Job_Growth_Projection

In [ ]:
# Kontingenčná tabuľka
cross_tab = pd.crosstab(df['AI_Adoption_Level'], df['Job_Growth_Projection'])
print("Kontingenčná tabuľka AI_Adoption_Level vs Job_Growth_Projection:\n", cross_tab)
# Heatmap
plt.figure(figsize=(9, 6))
sns.heatmap(cross_tab, annot=True, fmt='d', cmap='YlGnBu', linewidths=.5)
plt.title('Vzťah medzi adaptáciou AI a projekciou rastu pozícií')
plt.xlabel('Projekcia rastu pozícií')
plt.ylabel('Úroveň adaptácie AI')
plt.show()

Vzťah medzi Required_Skills a Salary_USD

In [ ]:

df_skills_exploded = df.assign(
    Required_Skill_Individual=df['Required_Skills'].str.lower().str.split(',\s*')
).explode('Required_Skill_Individual')

df_skills_exploded['Required_Skill_Individual'] = df_skills_exploded['Required_Skill_Individual'].str.strip()

# Odstránenie prázdne zručnosti
df_skills_exploded = df_skills_exploded[df_skills_exploded['Required_Skill_Individual'] != '']
df_skills_exploded = df_skills_exploded.dropna(subset=['Required_Skill_Individual'])


# Vypočet priemernej mzdy pre každú zručnosť
avg_salary_per_skill = df_skills_exploded.groupby('Required_Skill_Individual')['Salary_USD'].mean().reset_index()
avg_salary_per_skill = avg_salary_per_skill.sort_values(by='Salary_USD', ascending=False)
avg_salary_per_skill['Salary_USD'] = avg_salary_per_skill['Salary_USD'].round(2)

print("Priemerné mzdy pre top 10 zručností:")
print(avg_salary_per_skill.head(10))

# Vizualizácia Top 15 zručností podľa priemernej mzdy
top_n_salary_skills = 15 # Koľko zručností s najvyššou mzdou chcem zobraziť
most_lucrative_skills = avg_salary_per_skill.head(top_n_salary_skills)

plt.figure(figsize=(12, 8))
sns.barplot(
    data=most_lucrative_skills,
    x='Salary_USD',
    y='Required_Skill_Individual',
    palette='magma', # Iná paleta farieb
    edgecolor='black'
)
plt.title(f'Top {top_n_salary_skills} najlepšie platených zručností (Priemerná mzda)', fontsize=16)
plt.xlabel('Priemerná mzda (USD)', fontsize=12)
plt.ylabel('Zručnosť', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Vzťah medzi Automation_Risk (riziko automatizácie) a Job_Growth_Projection (projekcia rastu pozícií)

In [ ]:

# Vytvorenie kontingenčnej tabuľky
cross_tab_risk_growth = pd.crosstab(df['Automation_Risk'], df['Job_Growth_Projection'])
print("Kontingenčná tabuľka Automation_Risk vs Job_Growth_Projection:\n", cross_tab_risk_growth)
# Heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cross_tab_risk_growth, annot=True, fmt='d', cmap='Blues', linewidths=.5)
plt.title('Vzťah medzi rizikom automatizácie a projekciou rastu pozícií', fontsize=16)
plt.xlabel('Projekcia rastu pozícií', fontsize=12)
plt.ylabel('Riziko automatizácie', fontsize=12)
plt.show()

# 4. Zhrnutie kľúčových zistení a záver.

Cieľom projektu bolo preskúmať vplyv AI a automatizácie na rôzne pracovné pozície, odvetvia a zručnosti. Podarilo sa identifikovať, pozície ktoré  sú najviac ohrozené automatizáciou, a aké zručnosti sú žiadané v AI ére.

Kľúčové zistenia:

1. **Úroveň adaptácie AI v analyzovaných spoločnostiach:**

Analýza distribúcie úrovne adaptácie AI (Artificial Intelligence) vo firmách, zahrnutých v datasete, ukázala, že prevažná väčšina spoločností sa už aktívne podieľa na integrácii AI. Konkrétne, najvyšší podiel firiem spadá do kategórie strednej adaptácie AI (179 firiem), čo naznačuje aktívne experimentovanie a implementáciu AI riešení. Takmer rovnako silno je zastúpená nízka úroveň adaptácie (174 firem), čo môže predstavovať buď začiatočné štádiá implementácie, alebo pomalší prístup k novým technológiám. Vysoká úroveň adaptácie AI (147 firem), hoci je zastúpená o niečo menej, stále tvorí významnú časť firiem, ukazujúc lídrov v integrácii AI. Tieto dáta celkovo poukazujú na vzrastajúcu penetráciu AI do rôznych odvetví a jej postupnú, no zreteľnú adopciu na trhu práce.

2. **Riziko automatizácie a jeh ovplyv na pracovné pozície:**

Analýza úrovne rizika automatizácie odhalila, že pracovné pozície v datasete sú rozložené pomerne rovnomerne naprieč kategóriami nízkeho, stredného a vysokého rizika. Najväčší podiel pozícií  173 spadá do kategórie stredného rizika automatizácie, značný počet pozícií (169) vykazuje vysoké riziko automatizácie. Pozície s nízkym rizikom automatizácie  sú tiež podstatne zastúpené 158 pozícií. Toto rovnomerné rozdelenie naznačuje, že vplyv automatizácie je širokospektrálny a dotýka sa rôznych segmentov trhu práce.

**Detailnejší pohľad na top pracovné pozície v rámci každej kategórie rizika (ako je znázornené na grafe):**
*Nízke riziko automatizácie:* V tejto kategórii dominujú pozície ako Data Scientist, Cybersecurity Analyst a Operations Manager. To naznačuje, že role vyžadujúce komplexné analytické myslenie, špecializované zabezpečenie a strategické riadenie sú v súčasnosti menej náchylné na úplnú automatizáciu. Zaujímavé je, že aj HR Manager sa objavuje medzi pozíciami s nízkym rizikom, čo podčiarkuje dôležitosť ľudského faktora a interpersonálnych zručností.

*Stredné riziko automatizácie:* V kategórii stredného rizika sa výrazne presadzujú pozície ako UX Designer a Marketing Specialist. Tieto role pravdepodobne zažívajú čiastočnú automatizáciu rutinných úloh, no stále vyžadujú kreatívne myslenie a strategické rozhodovanie, ktoré je ťažké plne automatizovať. Je tu aj zastúpenie Data Scientist, čo môže naznačovať rôzne úrovne zamerania v rámci tejto širokej pozície.

*Vysoké riziko automatizácie*: Pozície s vysokým rizikom zahŕňajú najmä role ako HR Manager, Marketing Specialist a AI Researcher. Prítomnosť HR Manager a Marketing Specialist v tejto kategórii, napriek ich výskytu aj v nižších rizikách, môže poukazovať na podskupiny týchto úloh, ktoré sú náchylnejšie na automatizáciu, alebo na to, že hoci základné strategické úlohy zostávajú, rutinné operácie sú ohrozené. Pozícia AI Researcher s vysokým rizikom je na prvý pohľad prekvapivá, no môže to reflektovať, že aj v tejto oblasti sa objavujú automatizované nástroje pre repetitívne úlohy vo výskume.

Celkovo táto analýza zdôrazňuje, že riziko automatizácie nie je homogénne a líši sa aj v rámci podobných názvov pozícií, čo vyžaduje detailnejší pohľad na špecifické úlohy a zručnosti v rámci každej roly.

3. **Najžiadanejšie zručnosti a ich vplyv na odmeňovanie:**
Analýza požadovaných zručností na trhu práce, ovplyvnenom AI, odhalila jasné trendy v dopyte po špecifických expertízach. . **Medzi top 5 najčastejšie požadovaných zručností patria predovšetkým Python, Project Management, Cybersecurity, Machine Learning a Data Analysis**Toto naznačuje, že aj keď technické a dátovo orientované zručnosti dominujú, stále existuje silný dopyt po strategickom riadení a bezpečnostných aspektoch. Celkovo vysoký dopyt po týchto zručnostiach podčiarkuje posun smerom k dátovo riadeným rozhodnutiam, automatizácii a zároveň pretrvávajúcu dôležitosť efektívnej spolupráce a bezpečnosti v AI ére.

Ešte dôležitejšie je, že tieto kľúčové zručnosti sa priamo premietajú do finančného ohodnotenia. Graf porovnávajúci zručnosti s priemernými mzdami jasne ukazuje, že najvyššie priemerné mzdy sú spojené so zručnosťami ako Javascript  98 000,90 USD, Python 94 839.04 USD) a Sales 93101.25 USD. Zručnosti ako Machine Learning a Project Management tiež patria medzi špičku s priemernými mzdami nad 90 000 USD. Toto zistenie naznačuje, že investícia do rozvoja týchto špecifických, AI-relevantných a zároveň dôležitých technologických a obchodných zručností je kľúčová pre kariérny rast a finančnú prosperitu v meniacom sa prostredí trhu práce."




